In [3]:
# IMPORTAÇÃO DA LIBS PARA ETL 
import basedosdados as bd
import pandas as pd 
import numpy as np 

In [12]:
## IMPORTAÇÃO PARA A ANALISE GRAFICA DOS DADOS
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
py.init_notebook_mode(connected=True)
import seaborn as sns
from pandas_profiling import ProfileReport
import folium 
import seaborn as sbn
from folium.plugins import HeatMap
from folium import plugins

In [5]:
df = bd.read_sql(
    '''
    SELECT chmd.data_inicio, chmd.nome_unidade_organizacional, chmd.categoria, chmd.tipo, chmd.situacao,chmd.id_bairro, chmd.subtipo, br.nome as bairro, br.subprefeitura, chmd.latitude,chmd.longitude
    FROM datario.administracao_servicos_publicos.chamado_1746 chmd 
    INNER JOIN datario.dados_mestres.bairro br ON br.id_bairro = chmd.id_bairro
    WHERE DATE(data_inicio) = DATE('2023-04-01');
    ''',
    billing_project_id='case-escritorio-de-dados'
)

Downloading: 100%|██████████| 72/72 [00:00<00:00, 171.08rows/s]


In [6]:
display(df)

,data_inicio,nome_unidade_organizacional,categoria,tipo,situacao,id_bairro,subtipo,bairro,subprefeitura,latitude,longitude
0,2023-04-01 01:08:04,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Estacionamento Irregular,Encerrado,1,Fiscalização de estacionamento irregular de ve...,Saúde,Centro,NaN,NaN
1,2023-04-01 00:04:59,CCU - 1º Serviço de Fiscalização,Serviço,Comércio ambulante,Não Encerrado,14,Fiscalização de comércio ambulante,Santa Teresa,Centro,-22.921335,-43.186679
2,2023-04-01 00:38:22,CFE - Coordenação de Feiras,Serviço,Feiras,Não Encerrado,5,"Fiscalização/Inspeção de feira livre, feira de...",Centro,Centro,-22.912229,-43.188253
3,2023-04-01 00:14:00,CLF - Coordenadoria de Licenciamento e Fiscali...,Serviço,Ocupação de área pública,Não Encerrado,14,Fiscalização da ocupação de área pública por m...,Santa Teresa,Centro,-22.921123,-43.187288
4,2023-04-01 00:08:57,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Poluição sonora,Não Encerrado,14,Perturbação do sossego,Santa Teresa,Centro,-22.921276,-43.186388
...,...,...,...,...,...,...,...,...,...,...,...
67,2023-04-01 00:59:40,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Poluição sonora,Não Encerrado,144,Perturbação do sossego,Campo Grande,Zona Oeste,-22.879665,-43.563247
68,2023-04-01 00:07:07,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Poluição sonora,Não Encerrado,145,Perturbação do sossego,Senador Vasconcelos,Zona Oeste,-22.891154,-43.537462
69,2023-04-01 00:13:28,COMLURB - Companhia Municipal de Limpeza Urbana,Serviço,Manejo Arbóreo,Não Encerrado,144,Poda de árvore em logradouro,Campo Grande,Zona Oeste,NaN,NaN
70,2023-04-01 00:14:51,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Poluição sonora,Não Encerrado,148,Perturbação do sossego,Paciência,Zona Oeste,NaN,NaN


### 1) Quantos chamados foram abertos no dia 01/04/2023?

In [10]:
print('Os numeros de chamados abertos no dia 01-04-2023 foram {} casos'.format(df['id_bairro'].count()))

Os numeros de chamados abertos no dia 01-04-2023 foram 72 casos


### 2) Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

In [13]:
fig = px.histogram(df, x="tipo", barmode='group', text_auto= True)
fig.update_layout(bargap=0.2)
fig.show()

### 3) Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [75]:
maioresBairros = df.groupby(['bairro']).agg({
    'bairro': 'first',
    'longitude' :'first', 
    'latitude' :'first',
    'tipo':'count'
    }).sort_values(by='tipo',ascending=False)

In [76]:
bairro_com_mais_chamados = maioresBairros[maioresBairros['tipo'] >= 6 ]

In [59]:
fig = px.histogram(bairro_com_mais_chamados, x="bairro", y='tipo', barmode='group', text_auto= True)
fig.update_layout(bargap=0.2)
fig.show()

### 4) Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [14]:
fig = px.histogram(df, x="subprefeitura", barmode='group', text_auto= True)
fig.update_layout(bargap=0.2)
fig.show()

### 5) Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?
    /*como era um onibus e se tratava de um problema de ar condicionadoa a localização exata em si não ficou registrada

### Distribuição dos pontos de Chamada no dia 01/04/2023

In [60]:
mapa = folium.Map(location=[-22.9028, -43.2078],zoom_start=12.4)

In [65]:
# Estações iniciais 

df_lat_long_start = df[['latitude', 'longitude']].dropna()

dataStart = df_lat_long_start.values.tolist()
 
hmStart = HeatMap(dataStart,gradient={0.1: 'blue', 0.3: 'lime', 0.5: 'yellow', 0.7: 'orange', 1: 'red'}, 
                min_opacity=0.05, 
                max_opacity=0.9, 
                radius=25,
                use_local_extrema=False).add_to(mapa)

In [68]:
maioresBairrosMap = maioresBairros.dropna()
# MAKER POP'UPS DAS ESTAÇÕES
for _, r in maioresBairrosMap.iterrows():
    lat = r['latitude']
    lon = r['longitude']
    folium.Marker(location=[lat, lon],
                  popup=['{} <br> '.format(r['bairro']), '{} <br>'.format(r['tipo'])]).add_to(mapa)

mapa

### Extração dos dados com BigQuery e a Lib BasedosDados

In [150]:
df2 = bd.read_sql(
    '''
    SELECT chmd.id_chamado, chmd.tipo, chmd.subtipo, htl.evento, htl.data_inicial, htl.data_final
    FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos htl
    INNER JOIN datario.administracao_servicos_publicos.chamado_1746 chmd ON DATE(chmd.data_inicio) >= DATE(htl.data_inicial) AND DATE(chmd.data_inicio) <= DATE(htl.data_final) 
    WHERE chmd.subtipo = 'Perturbação do sossego' AND chmd.data_inicio BETWEEN '2022-01-01' AND '2023-12-31'
    ''',
    billing_project_id='case-escritorio-de-dados'
)

Downloading: 100%|██████████| 1212/1212 [00:00<00:00, 2740.70rows/s]


### 6) Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [71]:
print('Os numeros de chamados abertos com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 foram {} casos'.format(df2['id_chamado'].count()))

Os numeros de chamados abertos com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 foram 1212 casos


### 7) Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [96]:
numero_chamados = df2.groupby(['subtipo','evento']).agg({
    'evento': 'first',
    'tipo' :'first',
    'subtipo':'count'
    }).sort_values(by='tipo',ascending=False)

numero_chamados

evento             tipo  subtipo
subtipo                evento                                            
Perturbação do sossego Carnaval        Carnaval  Poluição sonora      241
                       Reveillon      Reveillon  Poluição sonora      137
                       Rock in Rio  Rock in Rio  Poluição sonora      834

### 8) Quantos chamados desse subtipo foram abertos em cada evento?

In [84]:
fig = px.histogram(numero_chamados, x="evento", y='subtipo', barmode='group', text_auto= True)
fig.update_layout(bargap=0.2)
fig.show()

### 9) Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [193]:
numero_chamados['media'] = round(numero_chamados['subtipo']/numero_chamados['subtipo'].sum()*100,2)

In [194]:
fig = px.histogram(numero_chamados, x="evento", y='media', barmode='group', text_auto= True)
fig.update_layout(bargap=0.2)
fig.show()

### 10) Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [196]:
media_carnaval = numero_chamados.iloc[0, 3]
media_reveillon = numero_chamados.iloc[1, 3]
media_rock_in_rio = numero_chamados.iloc[2, 3]
media_Total = round(len(df2) / ((pd.to_datetime('2023-12-31') - pd.to_datetime('2022-01-01')).days)*100,2)

In [199]:
medias_gerais = pd.DataFrame({
    'Chamados': ['Carnaval', 'Rock in Rio', 
              'Reveillon', 'Média Total'],
    'Médias': [media_carnaval, media_rock_in_rio,media_reveillon, media_Total]})

medias_gerais.sort_values(by='Médias', ascending=False)

,Chamados,Médias
3,Média Total,166.26
1,Rock in Rio,68.81
0,Carnaval,19.88
2,Reveillon,11.30
